In [3]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
from datetime import datetime, timedelta

In [6]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [7]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine)
# reflect the tables
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [8]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table
station = Base.classes.station
measurement = Base.classes.measurement

In [10]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

# Exploratory Precipitation Analysis

In [11]:
# Find the most recent date in the data set
most_recent_date = session.query(func.max(measurement.date)).scalar()

In [12]:
df.head()

NameError: name 'df' is not defined

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
most_recent_date = pd.to_datetime(most_recent_date)
twelve_months_ago = most_recent_date - pd.DateOffset(months=12)
twelve_months_ago = twelve_months_ago.to_pydatetime()

# Perform a query to retrieve the data and precipitation scores
results = session.query(measurement.date, measurement.prcp).filter(measurement.date >= twelve_months_ago).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df = pd.DataFrame(results, columns=['date', 'precipitation'])


# Sort the dataframe by date
df = df.sort_values('date')

# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x= 'date', y = 'precipitation', figsize=(10, 6))
plt.xlabel("Date")
plt.ylabel("Inches")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
total_station = session.query(station.station).count()
total_station

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_count = session.query(measurement.station, func.count(measurement.station)).group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()
station_count

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station_id = session.query(measurement.station, func.count(measurement.station)).group_by(measurement.station).order_by(func.count(measurement.station).desc()).first()[0]
lowest_temp = session.query(func.min(measurement.tobs)).filter(measurement.station == most_active_station_id).scalar()
highest_temp = session.query(func.max(measurement.tobs)).filter(measurement.station == most_active_station_id).scalar()
avg_temp = session.query(func.avg(measurement.tobs)).filter(measurement.station == most_active_station_id).scalar()
print(lowest_temp, highest_temp, avg_temp)

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

# Calculate the date 12 months ago from the most recent date
most_recent_date = session.query(func.max(measurement.date)).scalar()
most_recent_date = pd.to_datetime(most_recent_date)
twelve_months_ago = most_recent_date - pd.DateOffset(months=12)
twelve_months_ago = twelve_months_ago.to_pydatetime()

# Query temperature observation data for the last 12 months for the most active station
results = session.query(measurement.date, measurement.tobs).filter(measurement.station == most_active_station_id, measurement.date >= twelve_months_ago).all()

# Convert the query results into a pandas DataFrame
df = pd.DataFrame(results, columns=['date', 'temperature'])

# Plot the data as a histogram
plt.figure(figsize=(10, 6))
plt.hist(df['temperature'], bins=12, edgecolor='black')
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

# Close Session

In [15]:
def start_date(date):
    results = session.query(measurement.date, measurement.prcp).filter(measurement.date >= date).all()
    lowest_temp = session.query(func.min(measurement.tobs)).filter(measurement.date >= date).scalar()
    highest_temp = session.query(func.max(measurement.tobs)).filter(measurement.date >= date).scalar()
    avg_temp = session.query(func.avg(measurement.tobs)).filter(measurement.date >= date).scalar()
    return print(f"Lowest Temp:{lowest_temp}, Highest Temp:{highest_temp}, Avg Temp:{avg_temp}")

def end_date(date):
    results = session.query(measurement.date, measurement.prcp).filter(measurement.date <= date).all()
    lowest_temp = session.query(func.min(measurement.tobs)).filter(measurement.date <= date).scalar()
    highest_temp = session.query(func.max(measurement.tobs)).filter(measurement.date <= date).scalar()
    avg_temp = session.query(func.avg(measurement.tobs)).filter(measurement.date <= date).scalar()
    return print(f"Lowest Temp:{lowest_temp}, Highest Temp:{highest_temp}, Avg Temp:{avg_temp}")

In [16]:
start_date('2012-01-01')

Lowest Temp:53.0, Highest Temp:87.0, Avg Temp:73.29986460485998


In [63]:
def start_end_date(start, end):
    results = session.query(measurement.date, measurement.prcp).filter(measurement.date >= start, measurement.date <= end).all()
    lowest_temp = session.query(func.min(measurement.tobs)).filter(measurement.date >= start, measurement.date <= end).scalar()
    highest_temp = session.query(func.max(measurement.tobs)).filter(measurement.date >= start, measurement.date <= end).scalar()
    avg_temp = session.query(func.avg(measurement.tobs)).filter(measurement.date >= start, measurement.date <= end).scalar()
    return print (f"Lowest Temp: {lowest_temp}, Highest Temp: {highest_temp}, Avg Temp: {avg_temp}")

Lowest Temp: 56.0, Highest Temp: 86.0, Avg Temp: 72.26001511715798


In [16]:
# Close Session
session.close()